In [41]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

**Imported Dataset**

In [42]:
df=pd.read_csv("../input/gemstone-price-prediction/cubic_zirconia.csv")

In [43]:
df.describe()

In [44]:
df.info()

In [45]:
df

In [46]:
df.dtypes

**Data Preprocessing**

In [47]:
ndf=df.drop('Unnamed: 0', axis=1)
ndf

In [48]:
ndf.describe()

In [49]:
ndf.isna().sum()

In [50]:
ndf[ndf.isna().any(axis=1)]

- Depth attribute contains Null values that has to be removed or replaced before EDA.

In [51]:
fig, ax =plt.subplots(figsize =(10, 6))
sns.boxplot(ndf['depth'])

- Outliers are present in depth so for better results we can use the median value for treating the missing values.

In [52]:
ndf = ndf.fillna(ndf['depth'].median())

In [53]:
ndf.isna().sum()

In [54]:
ndf

In [55]:
print("Number of rows with x == 0: {} ".format((ndf.x==0).sum()))
print("Number of rows with y == 0: {} ".format((ndf.y==0).sum()))
print("Number of rows with z == 0: {} ".format((ndf.z==0).sum()))
print("Number of rows with depth == 0: {} ".format((ndf.depth==0).sum()))

- x,y,z and depth are dimensional values hence cubic zirconia diamonds having 0 for a particular dimension is meaningless for further EDA and modelling hence they must be removed or replaced by appropriate values.
- Replacing the values was found unsuitable for this particular case hence the rows are dropped which have zero dimensional attributes.

In [56]:
ndf = ndf.drop(ndf[ndf["x"]==0].index)
ndf = ndf.drop(ndf[ndf["y"]==0].index)
ndf = ndf.drop(ndf[ndf["z"]==0].index)

ndf.shape

In [57]:
ndf

In [58]:
dups = ndf.duplicated()
print('Number of duplicate rows = %d' % (dups.sum()))
print(ndf.shape)

In [59]:
ndf.drop_duplicates(inplace=True) 
ndf.shape

## Univariate Analysis of Numerical and Categorical Attributes

- Check for outliers
- Boxplots plotted for all attributes and the first 8 are closely observed to determine which among them are correlated with target label price.

In [60]:
cols = ['carat','depth', 'table', 'x', 'y', 'z','price' ]
for i in cols:
    sns.boxplot(ndf[i],whis=1.5)
    plt.show();

In [61]:
ndf.hist(figsize=(15,15))
plt.show()

In [62]:
plt.figure(figsize = (12,5))
sns.countplot('cut',data = ndf, order = ndf['cut'].value_counts().index)
plt.show()

In [63]:
plt.figure(figsize = (12,5))
sns.countplot('color',data = ndf, order = ndf['color'].value_counts().index)
plt.show()

In [64]:
plt.figure(figsize = (12,5))
sns.countplot('clarity',data = ndf, order = ndf['clarity'].value_counts().index)
plt.show()

## Bivariate Analysis

In [65]:
plt.figure(figsize=(20,10))
plt.subplot(131)
sns.boxplot(ndf['cut'], ndf['price'])

plt.subplot(132)
sns.boxplot(ndf['color'], ndf['price'])

plt.subplot(133)
sns.boxplot(ndf['clarity'], ndf['price'])

In [66]:
fig, ax = plt.subplots(figsize=(15,15))
sns.heatmap(ndf.corr(),cmap="RdPu",annot=True,ax=ax)

In [67]:
correlations = ndf.corr()
correlations["price"].sort_values(ascending=False)

- From the correlation heatmap we can observe strong correlation between the following attributes:
   #### - carat vs price
   #### - carat vs x,y,z
   #### - x,y,z vs price
   
   
- However we also observe that the table characteristic is not correlated with any of the numerical attributes.

In [68]:
sns.scatterplot(ndf['carat'], ndf['price'])
sns.set(rc={"figure.figsize":(10, 10)})
plt.title('carat vs price')
plt.show()

- From the scatterplot we observe that as carat increases price also witnesses a significant increase.

In [69]:
cols = ['carat', 'x', 'y', 'z', 'price']
df_ = ndf.loc[:,cols]
sns.pairplot(data =df_, diag_kind = 'kde', plot_kws = {"s": 1.5})

* From below pairplot we observe that carat and price, x and price and x and carat have a strong relation.

* In certain cases, even though the carat value is same, some diamonds have lesser prices while some have higher prices since there are other factors like x(length), y(width) and z(height) which are influencing the prices. 
